## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [25]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [26]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [27]:
# Set up data path
data_path = 'data'

In [28]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,8/15/20,Wards Island,12.70,1.0,36.0,1.60,7.94,Specialized
1,7/27/19,Wards Island,11.01,1.0,41.0,1.68,6.54,Specialized
2,6/30/20,Down Up LWS,14.95,1.0,48.0,1.80,8.31,Specialized
3,10/8/18,CP BL,7.55,NaN,52.0,0.87,8.71,Specialized
4,7/5/19,CP BL,6.76,NaN,52.0,0.87,7.80,Specialized
...,...,...,...,...,...,...,...,...
367,9/3/16,Down ES UP WS,16.72,1.0,52.0,1.87,8.96,Specialized
368,9/17/19,CP BL,7.31,NaN,59.0,0.98,7.43,Specialized
369,6/30/16,Citibike from Work,1.11,NaN,10.0,0.17,6.66,Citibike
370,8/6/16,Summer Streets,13.91,1.0,54.0,1.90,7.32,Specialized


In [29]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,8/15/20,Wards Island,12.70,1.0,36.0,1.60,7.94,Specialized
1,7/27/19,Wards Island,11.01,1.0,41.0,1.68,6.54,Specialized
2,6/30/20,Down Up LWS,14.95,1.0,48.0,1.80,8.31,Specialized
3,10/8/18,CP BL,7.55,0.0,52.0,0.87,8.71,Specialized
4,7/5/19,CP BL,6.76,0.0,52.0,0.87,7.80,Specialized
...,...,...,...,...,...,...,...,...
367,9/3/16,Down ES UP WS,16.72,1.0,52.0,1.87,8.96,Specialized
368,9/17/19,CP BL,7.31,0.0,59.0,0.98,7.43,Specialized
369,6/30/16,Citibike from Work,1.11,0.0,10.0,0.17,6.66,Citibike
370,8/6/16,Summer Streets,13.91,1.0,54.0,1.90,7.32,Specialized


In [30]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,8/15/20,Wards Island,12.70,1.0,36.0,1.60,7.94,Specialized
1,1,7/27/19,Wards Island,11.01,1.0,41.0,1.68,6.54,Specialized
2,2,6/30/20,Down Up LWS,14.95,1.0,48.0,1.80,8.31,Specialized
3,3,10/8/18,CP BL,7.55,0.0,52.0,0.87,8.71,Specialized
4,4,7/5/19,CP BL,6.76,0.0,52.0,0.87,7.80,Specialized
...,...,...,...,...,...,...,...,...,...
367,367,9/3/16,Down ES UP WS,16.72,1.0,52.0,1.87,8.96,Specialized
368,368,9/17/19,CP BL,7.31,0.0,59.0,0.98,7.43,Specialized
369,369,6/30/16,Citibike from Work,1.11,0.0,10.0,0.17,6.66,Citibike
370,370,8/6/16,Summer Streets,13.91,1.0,54.0,1.90,7.32,Specialized


In [31]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [32]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,12.70,1.60,7.94,Specialized
1,11.01,1.68,6.54,Specialized
2,14.95,1.80,8.31,Specialized
3,7.55,0.87,8.71,Specialized
4,6.76,0.87,7.80,Specialized
...,...,...,...,...
367,16.72,1.87,8.96,Specialized
368,7.31,0.98,7.43,Specialized
369,1.11,0.17,6.66,Citibike
370,13.91,1.90,7.32,Specialized


In [33]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [34]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [35]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [36]:
# Checking training data shape
X_train.shape

(279, 3)

In [37]:
# Checking testing data shape
X_test.shape

(93, 3)

In [38]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [39]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [40]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [41]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.978494623655914


In [42]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Specialized
2,Specialized
3,Specialized
4,Citibike
...,...
88,Citibike
89,Specialized
90,Specialized
91,Citibike


In [43]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
128,Specialized
254,Specialized
87,Specialized
85,Specialized
266,Citibike
...,...
107,Citibike
103,Specialized
286,Specialized
336,Citibike


In [44]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,128,Specialized
1,254,Specialized
2,87,Specialized
3,85,Specialized
4,266,Citibike
...,...,...
88,107,Citibike
89,103,Specialized
90,286,Specialized
91,336,Citibike


In [45]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,128,Specialized,Specialized
1,254,Specialized,Specialized
2,87,Specialized,Specialized
3,85,Specialized,Specialized
4,266,Citibike,Citibike
...,...,...,...
88,107,Citibike,Citibike
89,103,Specialized,Specialized
90,286,Specialized,Specialized
91,336,Citibike,Citibike


In [46]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
22,277,Specialized,Citibike
37,312,Specialized,Citibike


In [47]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,277,Specialized,Citibike,6/28/20,Uptown Trio,3.95,0.0,26.0,0.43,9.12,Specialized
1,312,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54,Specialized


In [48]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,277,Specialized,Citibike,6/28/20,Uptown Trio,3.95,0.0,26.0,0.43,9.12
1,312,Specialized,Citibike,5/25/20,Uptown Trio,4.27,0.0,34.0,0.57,7.54
